# Asset classes

This investigation continues the work done in the previous assignment.

## Part 1

This section reiterates the information about the datasets used.

### S&P 500 Index Total Returns

1. __Title:__ S&P 500 Total Returns (SP500)
2. __URL:__ [https://www.slickcharts.com/sp500/returns](https://www.slickcharts.com/sp500/returns)
3. __Source:__
   * __Source:__ Slickcharts
   * __Publisher:__ S&P Dow Jones Indices LLC
   * __Publication date:__ 2025-02-12
   * __Access date:__ 2025-02-13
4. __License:__ Standard & Poors Release
5. __Usable:__ Yes

### ICE BofA U.S. Corporate Index

1. __Title:__ ICE BofA US Corporate Index Total Return Index Value ([BAMLCC0A0CMTRIV](https://fred.stlouisfed.org/series/BAMLCC0A0CMTRIV))
2. __URL:__ [https://fred.stlouisfed.org/series/BAMLCC0A0CMTRIV](https://fred.stlouisfed.org/series/BAMLCC0A0CMTRIV)
3. __Source:__
   * __Source:__ FRED, the Federal Reserve Bank of St. Louis 
   * __Publisher:__ Ice Data Indices, LLC
   * __Publication date:__ 2025-02-12
   * __Access date:__ 2025-02-13
4. __License:__ ICE BofA Indices Release
5. __Usable:__ Yes

### S&P Case–Shiller Index

1. __Title:__ S&P CoreLogic Case-Shiller U.S. National Home Price Index ([CSUSHPINSA](https://fred.stlouisfed.org/series/CSUSHPINSA))
2. __URL:__ [https://fred.stlouisfed.org/series/CSUSHPINSA](https://fred.stlouisfed.org/series/CSUSHPINSA)
3. __Source:__
   * __Source:__ FRED, the Federal Reserve Bank of St. Louis
   * __Publisher:__ S&P Dow Jones Indices LLC
   * __Publication date:__ 2025-01-28
   * __Access date:__ 2025-02-13
4. __License:__ S&P CoreLogic Case-Shiller Home Price Indices Release
5. __Usable:__ Yes

### 10-year U.S. Treasury yield

1. __Title:__ Market Yield on U.S. Treasury Securities at 10-Year Constant Maturity, Quoted on an Investment Basis ([DGS10](https://fred.stlouisfed.org/series/DGS10))
2. __URL:__ [https://fred.stlouisfed.org/series/DGS10](https://fred.stlouisfed.org/series/DGS10)
3. __Source:__
   * __Source:__ FRED, the Federal Reserve Bank of St. Louis
   * __Publisher:__ Board of Governors of the Federal Reserve System (US)
   * __Publication date:__ 2025-02-12
   *  __Access date:__ 2025-02-13
4. __License:__ H.15 Selected Interest Rates Release
5. __Usable:__ Yes

### U.S. presidential party

1. __Title:__ List of presidents of the United States
2. __URL:__ [https://en.wikipedia.org/wiki/List_of_presidents_of_the_United_States](https://en.wikipedia.org/wiki/List_of_presidents_of_the_United_States)
3. __Source:__
   * __Source:__ Wikipedia
   * __Publisher:__ Wikimedia Foundation
   * __Publication date:__ 2025-02-13
   *  __Access date:__ 2025-02-13
4. __License:__ Creative Commons Attribution Share-Alike 4.0 license
5. __Usable:__ Yes

### Overview

* __Format:__ Comma-delimited (CSV)
* __Size:__ 2 KB
* __Observations:__ 38

### Sample

In [1]:
from itertools import islice

with open('data/raw/fredgraph.csv') as stream:
    for line in islice(stream, 0, 5):
        print(line, end="")

observation_date,BAMLCC0A0CMTRIV,DGS10,CSUSHPINSA,SP500,Party
1/1/1987,357.67,8.83,68.342,5.25,R
1/1/1988,392.57,9.14,73.277,16.61,R
1/1/1989,447.98,7.93,76.497,31.69,R
1/1/1990,481,8.08,75.972,-3.1,R


### Fields

* __Observation date__ (`observation_date`): `str`
* __ICE BofA U.S. Corporate Index__ (`BAMLCC0A0CMTRIV`): `Decimal`
* __10-year U.S. Treasury yield__ (`DGS10`): `Decimal`
* __S&P Case–Shiller Index__ (`CSUSHPINSA`): `Decimal`
* __S&P 500 Index returns__ (`SP500`): `Decimal`
* __ICE BofA U.S. Corporate Index returns__: `Decimal` (calculated column)
* __S&P Case–Shiller Index returns__: `Decimal` (calculated column)
* __U.S. presidential party__: `bool`

## Part 2

We can import these data into a dataframe.

In [2]:
from pandas import read_csv

table = read_csv('data/raw/fredgraph.csv')

table.head(5)

,observation_date,BAMLCC0A0CMTRIV,DGS10,CSUSHPINSA,SP500,Party
0,1/1/1987,357.67,8.83,68.342,5.25,R
1,1/1/1988,392.57,9.14,73.277,16.61,R
2,1/1/1989,447.98,7.93,76.497,31.69,R
3,1/1/1990,481.00,8.08,75.972,-3.10,R
4,1/1/1991,568.73,6.71,75.843,30.47,R


## Part 3

We will repeat part of our analysis using Pandas. We will normalize the data by parsing dates, expressing rates as decimals, and parsing the political party as an integer. We will also rename columns for brevity and clarity. Finally, we add columns for missing growth rates.

In [3]:
from pandas import set_option, to_datetime

table = table.rename(columns = {
    'observation_date': 'date',
    'BAMLCC0A0CMTRIV': 'debt_value',
    'DGS10': 'free_rate',
    'CSUSHPINSA': 'real_value',
    'SP500': 'equity_rate',
    'Party': 'party'
})

table['date'] = to_datetime(table['date'], format = '%m/%d/%Y')
table['free_rate'] = table['free_rate'] / 100
table['equity_rate'] = table['equity_rate'] / 100
table['debt_rate'] = table['debt_value'].pct_change()
table['real_rate'] = table['real_value'].pct_change()

set_option('future.no_silent_downcasting', True)

table['party'] = table['party'].replace({ 'D': 0, 'R': 1 })

table.head(5)

,date,debt_value,free_rate,real_value,equity_rate,party,debt_rate,real_rate
0,1987-01-01,357.67,0.0883,68.342,0.0525,1,NaN,NaN
1,1988-01-01,392.57,0.0914,73.277,0.1661,1,0.097576,0.072210
2,1989-01-01,447.98,0.0793,76.497,0.3169,1,0.141147,0.043943
3,1990-01-01,481.00,0.0808,75.972,-0.0310,1,0.073709,-0.006863
4,1991-01-01,568.73,0.0671,75.843,0.3047,1,0.182391,-0.001698


We will use these data to compute Sharpe ratios for the three asset classes. We will use arithmetic averages here.

In [12]:
free_rate = table['free_rate'].mean()

def sharpe(rate):
    return (table[rate].mean() - free_rate) / table[rate].std()

print("Sharpe ratio of equity:", sharpe('equity_rate'))
print("Sharpe ratio of debt:", sharpe('debt_rate'))
print("Sharpe ratio of real estate:", sharpe('real_rate'))

Sharpe ratio of equity: 0.4481999383144116
Sharpe ratio of debt: 0.28181579522678896
Sharpe ratio of real estate: -0.0026012943117476673


Based on this analysis, equity investments have the highest Sharpe ratio in terms of realized returns.